# Data Collection 

## 1. Firstly we collected ferilizer and yeild data from opendata pakistan  
https://opendata.com.pk/dataset/fertilizer-usage-production-per-acre-across-punjab-2002-2015  
`files/punjab_districts.csv`

In [1]:
import ee
import pandas as pd
import geopandas as gpd


### Data exploration for data source 1

In [4]:
d = pd.read_csv('punjab_districts.csv')
d.head()

,Year,Province,Division,District,Usage (in 1000 nutirient tons),Area Sown Total (Wheat),Area Sown (Rice),Area Sown (Cotton),Area Sown (Sugarcane),Wheat Production,Rice Production,Cotton Production,Sugarcane Production,Output/acre wheat,Output/Acre (Rice),Output/Acre (Cotton),Output/Acre (Sugarcane)
0,2002-03,Punjab,Bahawalpur Divn.,Bahawalpur,114,269,5.0,268.0,9.0,724,7,968.0,501,2.691450,1.400000,3.611940,55.666667
1,2002-03,Punjab,Bahawalpur Divn.,Bahawalnagar,93,282,41.0,178.0,27.0,695,65,544.0,1155,2.464539,1.585366,3.056180,42.777778
2,2002-03,Punjab,Bahawalpur Divn.,Rahim Yar Khan,168,303,13.0,307.0,43.0,740,19,1076.0,2620,2.442244,1.461538,3.504886,60.930233
3,2002-03,Punjab,D.G.Khan Divn.,D.G. Khan,48,156,27.0,93.0,3.0,407,53,411.0,142,2.608974,1.962963,4.419355,47.333333
4,2002-03,Punjab,D.G.Khan Divn.,Layyah,48,178,1.0,29.0,16.0,379,1,63.0,674,2.129213,1.000000,2.172414,42.125000


## 2. Secondly we get geojson data for boundaries of districts of punjab from kaggle  
https://www.kaggle.com/datasets/idrisonkaggle/pakistan-districts-and-province-boundaries?resource=download  
`files\pakistan_districts_province_boundries.geojson`

### Cleaning the geojson data to get the required data only for the districts of punjab

In [ ]:
import geopandas as gpd
import pandas as pd
import re

input_file = 'files/pakistan_districts_province_boundries.geojson' 

try:
    gdf = gpd.read_file(input_file)
    print(f"✅ Loaded GeoJSON. Total rows: {len(gdf)}")
    print(f"Columns found: {gdf.columns.tolist()}")
except Exception as e:
    print(f"❌ Error loading file: {e}")
    exit()

# 2. Filter for Punjab
province_col = 'NAME_1'

# Check if the column exists
if province_col not in gdf.columns:
    # Fallback search
    possible_cols = [c for c in gdf.columns if 'prov' in c.lower() or 'name_1' in c.lower()]
    if possible_cols:
        province_col = possible_cols[0]
    else:
        print("❌ Could not find Province column. Please check the column list printed above.")
        exit()

# Filter
punjab_gdf = gdf[gdf[province_col].astype(str).str.contains('Punjab', case=False, na=False)].copy()
print(f"✅ Filtered to Punjab. Districts found: {len(punjab_gdf)}")

# 3. Standardize District Names
district_col = 'NAME_3'

if district_col not in gdf.columns:
    possible_cols = [c for c in gdf.columns if 'dist' in c.lower() or 'name_3' in c.lower()]
    if possible_cols:
        district_col = possible_cols[0]
        

# Function to match your CSV style (removing 'District' word, extra spaces)
def clean_district_name(name):
    if not name: return ""
    name = str(name).strip()
    name = re.sub(r'District', '', name, flags=re.IGNORECASE) 
    return name.strip()

punjab_gdf['District_Name_Clean'] = punjab_gdf[district_col].apply(clean_district_name)

# 4. Save the Cleaned File
output_file = 'files/punjab_districts_cleaned.geojson'
punjab_gdf.to_file(output_file, driver='GeoJSON')
print(f"🎉 Success! Cleaned file saved as: {output_file}")

✅ Loaded GeoJSON. Total rows: 148
Columns found: ['objectid', 'province_territory', 'districts', 'shape_leng', 'shape_area', 'district_agency', 'status', 'cartodb_id', 'created_at', 'updated_at', 'geometry']
⚠️ 'NAME_1' not found. Guessing province column is: 'province_territory'
✅ Filtered to Punjab. Districts found: 36
⚠️ 'NAME_3' not found. Guessing district column is: 'districts'
🎉 Success! Cleaned file saved as: punjab_districts_cleaned.geojson
👉 USE THIS FILE IN YOUR GOOGLE EARTH ENGINE CODE NOW.


## 3. Collecting weather and climate data from google earth engine 

Authenticate with google engine app on google cloud console

In [ ]:
import ee
import os

# Delete old credentials if any
cred_path = os.path.expanduser('~/.config/earthengine/credentials')
if os.path.exists(cred_path):
    os.remove(cred_path)

# Force authentication with 'notebook' mode
ee.Authenticate(auth_mode='notebook', force=True)


Successfully saved authorization token.


In [ ]:
import os

name = os.environ.get('geeappname')
ee.Initialize(project=name)

In [ ]:
import ee
import geopandas as gpd
import pandas as pd
import json

try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

# 2. Load Cleaned GeoJSON
gdf = gpd.read_file('files/punjab_districts_cleaned.geojson')

# Ensure it is in WGS84 (Lat/Lon) and keep only necessary columns
if gdf.crs != 'EPSG:4326':
    gdf = gdf.to_crs('EPSG:4326')

# Keep only valid columns to avoid JSON errors
gdf = gdf[['District_Name_Clean', 'geometry']]

print(f"Loaded {len(gdf)} districts.")

# 3. Convert to GEE FeatureCollection
geojson_dict = json.loads(gdf.to_json())
districts_fc = ee.FeatureCollection(geojson_dict['features'])

# 4. Define NDVI Extraction Function
def get_ndvi(feature):
    district_name = feature.get('District_Name_Clean') 
    
    # Range of years: 2002 to 2015
    years = ee.List.sequence(2002, 2015) 
    
    def get_yearly_ndvi(y):
        y = ee.Number(y)
        # Season: Oct 1st of Year Y to March 31st of Year Y+1
        start = ee.Date.fromYMD(y, 10, 1)
        end = ee.Date.fromYMD(y.add(1), 3, 31)
        
        # MOD13A2: 16-Day NDVI
        ndvi = ee.ImageCollection('MODIS/006/MOD13A2') \
                 .filterDate(start, end) \
                 .select('NDVI') \
                 .mean() \
                 .clip(feature.geometry())
        
        # Reduce to a single number (Mean NDVI for the district)
        # We use 'bestEffort=True' to avoid "Image too large" errors
        mean_val = ndvi.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=feature.geometry(),
            scale=1000,
            bestEffort=True
        ).get('NDVI')
        
        return ee.Feature(None, {
            'District': district_name,
            'Year': y,
            'Mean_NDVI': mean_val
        })
    
  
    return ee.FeatureCollection(years.map(get_yearly_ndvi))

# 5. Run Extraction (Server Side)
print("Sending request to Google Earth Engine... (This may take 1-2 mins)")

results = districts_fc.map(get_ndvi).flatten()

try:
    data = results.getInfo()['features']
    ndvi_list = [d['properties'] for d in data]

    df_ndvi = pd.DataFrame(ndvi_list)

    # Scale NDVI (MODIS is 0-10000, we want 0-1)
    if not df_ndvi.empty:
        df_ndvi['Mean_NDVI'] = df_ndvi['Mean_NDVI'] / 10000.0
        df_ndvi.to_csv('files/punjab_ndvi_2002_2015.csv', index=False)
        print(" Success! NDVI data saved to 'files/punjab_ndvi_2002_2015.csv'")
        print(df_ndvi.head())
    else:
        print(" Error: Resulting DataFrame is empty. Check your District Names.")

except Exception as e:
    print(f" Error extracting data: {e}")

Loaded 36 districts.
Sending request to Google Earth Engine... (This may take 1-2 mins)
✅ Success! NDVI data saved to 'punjab_ndvi_2002_2015.csv'
  District  Mean_NDVI  Year
0  Chiniot   0.451566  2002
1  Chiniot   0.449537  2003
2  Chiniot   0.465009  2004
3  Chiniot   0.435443  2005
4  Chiniot   0.477645  2006


### Data integrations: Calculating geometric centroids for each district from the GeoJSON file and automating the retrieval of daily historical weather data via the NASA POWER API, which is then aggregated into yearly total rainfall and average temperature features.

In [ ]:
import geopandas as gpd
import pandas as pd
import requests
import time
import sys

# 1. Load GeoJSON to get Coordinates
try:
    gdf = gpd.read_file('punjab_districts_cleaned.geojson')
    
    # Ensure it is in WGS84 (Lat/Lon)
    if gdf.crs != 'EPSG:4326':
        gdf = gdf.to_crs('EPSG:4326')


    gdf_projected = gdf.to_crs('EPSG:3857') 
    gdf['centroid'] = gdf_projected.geometry.centroid.to_crs('EPSG:4326')
    
    # Create a list of districts with their Lat/Lon
    districts_info = []
    for idx, row in gdf.iterrows():
        # Try to find the district name column
        d_name = row.get('District_Name_Clean', row.get('District', row.get('NAME_3', f"District_{idx}")))
        
        districts_info.append({
            'District': d_name,
            'Lat': round(row.centroid.y, 4),
            'Lon': round(row.centroid.x, 4)
        })

    print(f"✅ Loaded {len(districts_info)} districts for weather fetching.")

except Exception as e:
    print(f" Error loading GeoJSON: {e}")
    sys.exit()

# 2. Define NASA POWER API Function
def fetch_weather(lat, lon, start_year=2002, end_year=2015):
    base_url = "https://power.larc.nasa.gov/api/temporal/daily/point"
    
    params = {

        'parameters': 'PRECTOTCORR,T2M', 
        'community': 'AG',
        'longitude': lon,
        'latitude': lat,
        'start': f"{start_year}0101",
        'end': f"{end_year}1231",
        'format': 'JSON'
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"⚠️ API Error {response.status_code}: {response.text}")
        return None

# 3. Loop and Download
weather_data = []

print(f" Starting NASA POWER downloads for {len(districts_info)} districts...")


for i, dist in enumerate(districts_info):
    name = dist['District']
    print(f"[{i+1}/{len(districts_info)}] Fetching climate for: {name}...")
    
    data = fetch_weather(dist['Lat'], dist['Lon'])
    
    if data:
        try:
            # Parse the JSON response
            properties = data['properties']['parameter']
            
            # Note: NASA uses the exact parameter name in the response
            rain = properties['PRECTOTCORR'] 
            temp = properties['T2M']
            
            # Aggregate Daily Data to Yearly
            for year in range(2002, 2016):
                year_str = str(year)
                
                # Filter values for the specific year
                yearly_rain_vals = [val for date, val in rain.items() if date.startswith(year_str) and val != -999.0]
                yearly_temp_vals = [val for date, val in temp.items() if date.startswith(year_str) and val != -999.0]
                
                if yearly_rain_vals:
                    total_rain = sum(yearly_rain_vals)
                    avg_temp = sum(yearly_temp_vals) / len(yearly_temp_vals)
                    
                    weather_data.append({
                        'District': name,
                        'Year': year,
                        'Total_Rainfall_mm': round(total_rain, 2),
                        'Avg_Temp_C': round(avg_temp, 2)
                    })
        except KeyError as e:
            print(f" Error parsing data for {name}: {e}")
        
    time.sleep(1.0)

# 4. Save to CSV
if weather_data:
    df_weather = pd.DataFrame(weather_data)
    output_filename = 'files/punjab_climate_2002_2015.csv'
    df_weather.to_csv(output_filename, index=False)
    print(f"\n🎉 Success! Climate data saved to '{output_filename}'")
    print(df_weather.head())
else:
    print("\n Failed to collect weather data.")

✅ Loaded 36 districts for weather fetching.
🚀 Starting NASA POWER downloads for 36 districts...
⏳ This might take 2-3 minutes. Please wait...
[1/36] Fetching climate for: Chiniot...
[2/36] Fetching climate for: Lahore...
[3/36] Fetching climate for: Bhakkar...
[4/36] Fetching climate for: Gujrat...
[5/36] Fetching climate for: Mandi Bahauddin...
[6/36] Fetching climate for: Hafizabad...
[7/36] Fetching climate for: Khushab...
[8/36] Fetching climate for: Attock...
[9/36] Fetching climate for: Jhang...
[10/36] Fetching climate for: Bahawalnagar...
[11/36] Fetching climate for: Bahawalpur...
[12/36] Fetching climate for: Chakwal...
[13/36] Fetching climate for: Dera Ghazi Khan...
[14/36] Fetching climate for: Faisalabad...
[15/36] Fetching climate for: Gujranwala...
[16/36] Fetching climate for: Jhelum...
[17/36] Fetching climate for: Kasur...
[18/36] Fetching climate for: Khanewal...
[19/36] Fetching climate for: Layyah...
[20/36] Fetching climate for: Lodhran...
[21/36] Fetching climat

## Data cleaning : Standardizing col names

In [ ]:
import pandas as pd
import numpy as np
import re

raw_file_name = 'files/punjab_districts.csv' 

try:
    df = pd.read_csv(raw_file_name)
    print(f"Loaded original file: {raw_file_name}")
except FileNotFoundError:
    print(f"❌ ERROR: Could not find '{raw_file_name}'. Please rename the variable above.")
    exit()


df.columns = [
    'Year_Range', 'Province', 'Division', 'District', 'Fertilizer_Usage_K_Tons',
    'Area_Sown_Wheat', 'Area_Sown_Rice', 'Area_Sown_Cotton', 'Area_Sown_Sugarcane',
    'Production_Wheat_Tons', 'Production_Rice_Tons', 'Production_Cotton_Tons', 
    'Production_Sugarcane_Tons', 'Yield_Wheat_Acre', 'Yield_Rice_Acre', 
    'Yield_Cotton_Acre', 'Yield_Sugarcane_Acre'
]


df['Year'] = df['Year_Range'].astype(str).str.split('-').str[0].astype(int)

# Fix District Names (Remove 'Divn.', extra spaces)
def clean_name(name):
    if pd.isna(name): return name
    name = str(name).strip()
    name = re.sub(r'\s*Divn\.', '', name, flags=re.IGNORECASE) # Remove "Divn."
    name = re.sub(r'[^a-zA-Z\s]', '', name) # Remove special chars
    return name.strip()

df['District'] = df['District'].apply(clean_name)

# --- 4. Save the V2 File ---
output_name = 'files/cleaned_punjab_agri_2002_2015_V2.csv'
df.to_csv(output_name, index=False)

print(f" Success! Generated: {output_name}")
print(f"   Rows: {len(df)}")


Loaded original file: punjab_districts.csv
✅ Success! Generated: cleaned_punjab_agri_2002_2015_V2.csv
   Rows: 416
👉 Now run the Health Check script again.


## Data integrity verification

In [ ]:
import pandas as pd
import os

# Define your file names
files = {
    "Agri": "files/cleaned_punjab_agri_2002_2015_V2.csv",
    "NDVI": "files/punjab_ndvi_2002_2015.csv",
    "Climate": "files/punjab_climate_2002_2015.csv"
}

print(" DATA HEALTH CHECK REPORT\n" + "="*30)

success_count = 0

for name, filename in files.items():
    print(f"\nchecking: {name} Data ({filename})...")
    
    if os.path.exists(filename):
        try:
            df = pd.read_csv(filename)
            
            # 1. Check Row Count
            print(f"    Found! Rows: {len(df)}, Columns: {len(df.columns)}")
            
            # 2. Check Key Columns for Merging
            cols = [c.lower() for c in df.columns]
            has_district = any('dist' in c for c in cols)
            has_year = any('year' in c for c in cols)
            
            if has_district and has_year:
                print("    Merge Keys (District, Year) present.")
                success_count += 1
            else:
                print(f"    CRITICAL WARNING: Missing 'District' or 'Year' column! Found: {df.columns.tolist()}")
            
            # 3. Show a sneak peek
            print(f"    Columns: {df.columns.tolist()}")
            
        except Exception as e:
            print(f"    Error reading file: {e}")
    else:
        print(f"    FILE MISSING: The file '{filename}' was not found in this folder.")

print("\n" + "="*30)
if success_count == 3:
    print("  all 3 datasets VERIFIED. We can merge now.")
else:
    print(f" WAIT! Only {success_count}/3 datasets are ready. Fix the missing/broken ones first.")

📊 DATA HEALTH CHECK REPORT

checking: Agri Data (cleaned_punjab_agri_2002_2015_V2.csv)...
   ✅ Found! Rows: 416, Columns: 18
   ✅ Merge Keys (District, Year) present.
   👀 Columns: ['Year_Range', 'Province', 'Division', 'District', 'Fertilizer_Usage_K_Tons', 'Area_Sown_Wheat', 'Area_Sown_Rice', 'Area_Sown_Cotton', 'Area_Sown_Sugarcane', 'Production_Wheat_Tons', 'Production_Rice_Tons', 'Production_Cotton_Tons', 'Production_Sugarcane_Tons', 'Yield_Wheat_Acre', 'Yield_Rice_Acre', 'Yield_Cotton_Acre', 'Yield_Sugarcane_Acre', 'Year']

checking: NDVI Data (punjab_ndvi_2002_2015.csv)...
   ✅ Found! Rows: 504, Columns: 3
   ✅ Merge Keys (District, Year) present.
   👀 Columns: ['District', 'Mean_NDVI', 'Year']

checking: Climate Data (punjab_climate_2002_2015.csv)...
   ✅ Found! Rows: 504, Columns: 4
   ✅ Merge Keys (District, Year) present.
   👀 Columns: ['District', 'Year', 'Total_Rainfall_mm', 'Avg_Temp_C']

🎉 ALL SYSTEMS GO! You have all 3 datasets. We can merge now.


# Data merging

In [ ]:
import pandas as pd
import re

# 1. Load the 3 Datasets
try:
    df_agri = pd.read_csv("cleaned_punjab_agri_2002_2015_V2.csv")
    df_ndvi = pd.read_csv("punjab_ndvi_2002_2015.csv")
    df_climate = pd.read_csv("punjab_climate_2002_2015.csv")
    print("✅ All 3 files loaded successfully.")
except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    exit()

# 2. Standardize Keys (District Name Cleaning)
def standardize_name(name):
    name = str(name).lower().strip()
    name = re.sub(r'district', '', name) # Remove 'district' word
    name = re.sub(r'[^a-z]', '', name)   # Keep only letters
    return name

# Apply to all 3 dataframes
df_agri['Merge_Key'] = df_agri['District'].apply(standardize_name)
df_ndvi['Merge_Key'] = df_ndvi['District'].apply(standardize_name)
df_climate['Merge_Key'] = df_climate['District'].apply(standardize_name)

# 3. Perform the Merge
# Merge Agri + NDVI
df_merged = pd.merge(df_agri, df_ndvi, on=['Merge_Key', 'Year'], how='inner')

# Merge Result + Climate
df_final = pd.merge(df_merged, df_climate, on=['Merge_Key', 'Year'], how='inner')

# 4. Clean Up
# Drop extra columns (like duplicated District names from merge)
# We keep the original 'District_x' as the main District name
df_final = df_final.rename(columns={'District_x': 'District'})
cols_to_drop = [c for c in df_final.columns if 'District_y' in c or 'Merge_Key' in c]
df_final = df_final.drop(columns=cols_to_drop)

# 5. Save Master Dataset
output_file = "final_dataset_ml_ready.csv"
df_final.to_csv(output_file, index=False)

print("\n" + "="*40)
print(f" MASTER DATASET CREATED: {output_file}")
print(f"   Total Rows: {len(df_final)}")
print(f"   Features: {df_final.columns.tolist()}")
print("="*40)
print(" Data ready for the ML Pipeline!")

✅ All 3 files loaded successfully.

🎉 MASTER DATASET CREATED: final_dataset_ml_ready.csv
   Total Rows: 392
   Features: ['Year_Range', 'Province', 'Division', 'District', 'Fertilizer_Usage_K_Tons', 'Area_Sown_Wheat', 'Area_Sown_Rice', 'Area_Sown_Cotton', 'Area_Sown_Sugarcane', 'Production_Wheat_Tons', 'Production_Rice_Tons', 'Production_Cotton_Tons', 'Production_Sugarcane_Tons', 'Yield_Wheat_Acre', 'Yield_Rice_Acre', 'Yield_Cotton_Acre', 'Yield_Sugarcane_Acre', 'Year', 'Mean_NDVI', 'District', 'Total_Rainfall_mm', 'Avg_Temp_C']
👉 You are now ready for the ML Pipeline!
